Learning Automaton
Forum Scrape Project
Information Processing


Goals:
Associate words to adjectives via rules
Associate words to adjectives by sentence
Associate user to Nouns
Instances of fruits, veggies
do basic sentiment analasys using 1, 2, 3, depending on effectivness

In [1]:
fruit_match_words = set(['pulls','fruit','apple','apricot','banana','cantaloupe','cherry','date','citrus','avocado','carissa','guava','cherry','citron','clementine','crabapple','grape','grapefruit','honeydew','lemon','lime','orange','mandarin','mango','papaya','peach','pear','pineapple','plantain','plum','pomelo','tangarine','watermelon'])
vegetable_match_words =  set(['vegetable','artichoke','eggplant','asparagu','broccoli','cabbage','cauliflower','celery','spinach','lettuce','onion','beet','carrot','potato','yam','turnip','squash','tomato','watercress'])

In [ ]:
import pymongo

from pprint import pprint
from pymongo import MongoClient
client = MongoClient('192.168.2.70', 27017)
db=client['fruitsandveggies']
collection = db['aggregate-posts-out_3']

In [2]:
import spacy
from spacy import displacy

# TODO test lg vs sm
# TODO test spacy core in binder

#nlp_md = spacy.load('en_core_web_md')
nlp_lg = spacy.load('en_core_web_lg')


In [94]:
nlp_sm = spacy.load('en_core_web_sm')

In [95]:
nlp = nlp_sm

In [ ]:
for word in nlp.Defaults.stop_words:
    lex = nlp.vocab[word]
    lex.is_stop = True

In [12]:
##--NEED TO INSTALL SPACY 2.1 TO USE MATCH ON LEMMA--##


from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

class EntityMatcher(object):

    def __init__(self, nlp, terms, matcher_name, label):
        patterns = [nlp(text) for text in terms]
        self.matcher = PhraseMatcher(nlp.vocab, attr="LEMMA")
        self.matcher.add(label, None, *patterns)
        self.name = matcher_name

    def __call__(self, doc):
        matches = self.matcher(doc)
        for match_id, start, end in matches:
            span = Span(doc, start, end, label=match_id)
            doc.ents = list(doc.ents) + [span]
        return doc

In [122]:
fruit_matcher = EntityMatcher(nlp, fruit_match_words, "fruit_matcher", "FRUIT")

vegetable_matcher = EntityMatcher(nlp, vegetable_match_words, "vegetable_matcher", "VEGETABLE")

#nlp.disable_pipes("ner")
#nlp.add_pipe(fruit_matcher)
#nlp.add_pipe(vegetable_matcher)

#doc_strip = nlp(' '.join(doc))
nlp.pipeline


[('tagger', <spacy.pipeline.pipes.Tagger at 0x1a405d0b6a0>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x1a407a2fd68>),
 ('fruit_matcher', <__main__.EntityMatcher at 0x1a405c50278>),
 ('vegetable_matcher', <__main__.EntityMatcher at 0x1a405c50240>)]

In [19]:
raw = "Apples are the best. Us enlightened who call them, 'pulls love them more than any other fruit. 'Pulls are tasty, sweet, and crisp. All other fruits and those yucky vegetables are inferior. Discuss.. Ya, 'pulls are the best. 'Pulls are ripe, juicy, and taste sooooo good. Nothing else can match! I'd eat apples every day all day if I could, doctors be damned!. Yall get real. Apples are gross. The worst of the fruits, which are all bad. Everyone knows that veggies are powerfood. Get your weak apple sauce out of here.. Everyone knows grapes are superior you filthy casual. Enjoy your cyanide balls. Battlestar Galactica.. NO ONE READ THE ABOVE. THAT IS NOT ME.  Identity theft is not a joke, Jim! Millions of families suffer every year!. MICHAEL!  *storms off*. Oh, that's funny.  MICHAEL. From my head tomatoes.. Never Leaf Me.  We make a pearfect couple.. Peaches are better than all others. Peaches are the best, cabbage is the worst, end of story.  Smelly Cabbage; yuck. Corn cob rows!  Take me home,  to the placeeee,  I belonggg!!!!  WEST INDIANAAAAA  BUTTER MAMAAAAA  TAKE ME HOMEEEEE  CORN BOB ROWWWSS. If a squash can make you smile.... Lol, tomatoes and squash are both fruits! Get this compost out of the veggie master plate forum. Na, tomatoes and squash are fruits in name only. Acidic tomatoes and dirty squash, just like dirty tomatoes, should stay out of our sweet fruit forum. Like yucky lettuce, they belong in the vegetable forum, aka the compost.. Knock Knock. Who's There?. Banana. Banana who?. Knock Knock.... Who's There?... Banana. Banana who?. Knock Knock. Whos. ... There..... Orange!. Orange who?. ORANGE YOU GLAD I DIDN'T SAY BANANA??!?. You thought you'd pull a fast one on me, you little bitch? I'll have you know I graduated top of my class in the Tasty Peels, and I've been involved in numerous secret raids on Al-Quinoa, and I have over 300 confirmed spills. I am trained in vanilla warfare and I'm the top ripener in the entire US farmed forces. You are nothing to me but just another target grocery aisle. I will wipe you the fuck out with precision the likes of which has never been seen before on this Earth, mark my fucking words. You think you can get away with saying that shit to me over the Internet? Think again, fucker. As we speak I am contacting my secret network of pies across the USA and your IP is being traced right now so you better prepare for the storm, maggot. The storm that wipes out the pathetic little thing you call your life. You're fucking dead, kid. I can be anywhere, anytime, and I can eat you in over seven hundred ways, and that's just with my bare hands. Not only am I extensively trained in organic combat, but I have access to the entire pesticide arsenal of the United States Soybean Crops and I will use it to its full extent to wipe your miserable ass off the face of the continent, you little shit. If only you could have known what appleholey retribution your little \"clever\" comment was about to bring down upon you, maybe you would have held your fucking tongue. But you couldn't, you didn't, and now you're paying the price, you goddamn idiot. I will shit fury all over you and you will drown in it. Your lettuce head is fucking dead, kiddo.. Get boiled, damn."
#raw = "apple. bananas. cars. turtles. carrot."


In [102]:

keep_tokens = [token.text for token in doc if not( # Iterating over a spaCy Doc returns the tokens within the Doc 
        token.is_stop or # remove stop words, or words that arn't useful for most NLP problems
        (token.pos_ in ['PROPN','NUM','SYM','X','PUNCT'] and not token.text==".") or # remove proper nouns, numbers, symbols, unknowns, and all punctuation except periods, so we can delineate sentences.
        #token.is_oov or # Remove words that are out of vocabulary (oov). We might lose some useful data, however, to be safe we take it out as we can't identify it.
        token.is_space # Remove blank sentences
    )]
#    print(post.thread_title,keep_tokens) # will print once for each post, so there will be duplicates


doc_strip = nlp(' '.join(keep_tokens))


In [97]:
doc_full = nlp(raw)

In [103]:
doc = doc_full

In [104]:
from collections import Counter
fruits = Counter()
veggies = Counter()
for ent in doc.ents:
    (fruits if ent.label_ == "FRUIT" else veggies).update([ent.lemma_])
print(fruits)
print(veggies)

Counter({'fruit': 6, 'pull': 5, 'apple': 4, 'peach': 2, 'grape': 1})
Counter({'squash': 4, 'tomato': 4, 'vegetable': 2, 'lettuce': 2, 'cabbage': 1})


In [ ]:
for token in doc_strip:
   print('{}, {}, {}'.format(token.text, token.lemma_, token.norm_, token.pos_))

In [105]:
doc = doc_strip
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

noun_is_adj_plus = [{"ENT_TYPE": {"IN": ["FRUIT","VEGETABLE"]}}, {"POS": "ADV", "OP": "*"}, {"POS": "DET", "OP": "*"},
            {"POS": "ADJ", "OP": "+"}]\

noun_is_adj = [{"ENT_TYPE": {"IN": ["FRUIT","VEGETABLE"]}}, {"POS": "ADJ"}]
noun_is_adv = [{"ENT_TYPE": {"IN": ["FRUIT","VEGETABLE"]}}, {"POS": "ADV"}]

adj_noun = [ {"POS": "ADJ"},{"ENT_TYPE": {"IN": ["FRUIT","VEGETABLE"]}}]

matcher.add("NounIsAdj", None, noun_is_adj)  # add pattern
matcher.add("NounIsAdv", None, noun_is_adv)  # add pattern
matcher.add("AdjNoun", None, adj_noun)

matches = matcher(doc)

fruit_words = []
veggie_words = []

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    #if not set([token.lemma_ for token in span]).isdisjoint(fruit_match_words):
    #    print(span.text)
    
        
    #if set(span.text.split(' ')) & fruit_match_words:
    #    print(span.text)

    print(match_id, string_id, start, end, span.text)
print(len(matches))

9076452796594085796 NounIsAdv 0 2 Apples best
16820485401210444574 NounIsAdj 8 10 Pulls tasty
16820485401210444574 NounIsAdj 13 15 fruits yucky
6136520347023808161 AdjNoun 14 16 yucky vegetables
16820485401210444574 NounIsAdj 15 17 vegetables inferior
9076452796594085796 NounIsAdv 19 21 pulls best
16820485401210444574 NounIsAdj 22 24 Pulls ripe
16820485401210444574 NounIsAdj 37 39 Apples gross
6136520347023808161 AdjNoun 40 42 worst fruits
16820485401210444574 NounIsAdj 41 43 fruits bad
16820485401210444574 NounIsAdj 52 54 grapes superior
9076452796594085796 NounIsAdv 80 82 Peaches better
16820485401210444574 NounIsAdj 83 85 Peaches best
6136520347023808161 AdjNoun 84 86 best cabbage
16820485401210444574 NounIsAdj 85 87 cabbage worst
6136520347023808161 AdjNoun 118 120 Acidic tomatoes
6136520347023808161 AdjNoun 123 125 dirty tomatoes
6136520347023808161 AdjNoun 126 128 sweet fruit
6136520347023808161 AdjNoun 131 133 yucky lettuce
9076452796594085796 NounIsAdv 152 154 pull fast
20


In [107]:
doc = doc_strip
from collections import defaultdict
word_count = defaultdict(int)
for token in doc:
    if token.lemma_ in fruit_match_words.union(vegetable_match_words):
        word_count[token.lemma_] += 1

fruit_count = sum([word_count[key] for key in word_count.keys() if key in fruit_match_words])
veg_count = sum([word_count[key] for key in word_count.keys() if key in vegetable_match_words])

print("Fruit mentions: {}".format(fruit_count))
print("Veg mentions: {}".format(veg_count))

Fruit mentions: 13
Veg mentions: 14


In [120]:
#posts = collection.find({})
doc = doc_strip
noun_adj_pairs = []

for sent in doc.sents:
    for i,token in enumerate(sent):
        if token.ent_type_ not in ('FRUIT','VEGETABLE'):
            continue
        for j in range(i+1,len(sent)):
            if (sent[j].pos_ == 'ADJ'):
                noun_adj_pairs.append((token,sent[j]))
                break
    
fruit_pairs = [(pair[0].lemma_,pair[1].lemma_) for pair in noun_adj_pairs if pair[0].lemma_ in fruit_match_words]
veg_pairs = [(pair[0].lemma_,pair[1].lemma_) for pair in noun_adj_pairs if pair[0].lemma_ in vegetable_match_words]

print(fruit_pairs)
print(veg_pairs)

[('fruit', 'yucky'), ('apple', 'real'), ('apple', 'gross'), ('fruit', 'bad'), ('apple', 'superior'), ('grape', 'superior'), ('peach', 'good'), ('fruit', 'veggie')]
[('vegetable', 'inferior'), ('cabbage', 'bad'), ('tomato', 'dirty'), ('squash', 'dirty'), ('tomato', 'sweet'), ('lettuce', 'dead')]
